In [1]:
require "hdf5"
require "optim"
require "nn"

In [2]:
function capitalization(caps)
    if caps == false then
        X_train = X_train:sub(1, X_train:size(1), 1, X_train:size(2)/2)
        X_valid = X_valid:sub(1,X_valid:size(1), 1, X_valid:size(2)/2)
        X_test = X_test:sub(1, X_test:size(1), 1, X_test:size(2)/2)
    end
    return X_train, X_valid, X_test
end

--ensures that the indexes are changed to match window size
function changeIndex(X, caps, window_size)
    if caps then
        for i=1,X:size(2)/2 do
            X:select(2,i):add(1+(i-1)*N)
        end
    else
        for i=1,X:size(2) do
            X:select(2,i):add(1+(i-1)*N)
        end
    end
    if caps then
        for i=(X:size(2)/2 + 1),X:size(2) do
            local idx = (i-(X:size(2)/2))+(i-(X:size(2)/2 + 1))
            X:select(2,i):add(N*window_size + idx)
        end
    end
    return X
end

function onehotencode(classes, target)
    onehot = torch.zeros(target:size(1), classes)
    for i=1,target:size(1) do
        onehot[i][target[i]] = 1
    end
    return onehot
end

function lookupweights(X, Y, caps, window_size)
    local add_col = 0
    if caps==true then
        add_col = window_size*2
    end
    W = torch.zeros(45, window_size*(N)+add_col)
    for i=1,X:size(1) do
        for j=1,X:size(2) do
            local row = Y[i]
            local column = X[i][j]
            W[row][column] = W[row][column] + 1
        end
    end
    return W
end

In [3]:
function NB(Y,fc, alpha)
    --calculate log posterior
    local cc = torch.zeros(1, Y:size(2))
    --local fc = torch.zeros(Y:size(2), X:size(2))
    local clp = torch.zeros(1, Y:size(2))

    --collapse column wise so Nx1
    cc:add(Y:sum(1))
    --THIS IS ALPHA
    --smoothing
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    local total = cc:sum(2)
    cc:div(total[1][1])
    cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), cc
end

function predict(X, W, b)
  local jll = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    jll[i] = W:index(2, X[i]):sum(2)
    --right now prior is messed up
    jll[i]:add(b)
  end
  return jll
end

function predict_score()
    local c = 0
    for i=1,indices_pred:size(1) do
        if indices_pred[i][1] == indices_true[i][1] then
            c = c + 1
        end
    end
    return c/Y_valid:size()[1]
end

function write2file(fname, pred) 
    f = io.open(fname, "w")
    f:write("ID,Class\n")
    for i=1,pred:size(1) do
        f:write(tostring(i) .. "," .. tostring(pred[i][1]) .. "\n")
    end
    f:close()
end

In [15]:
--read in files
f = hdf5.open("data.hdf5", "r")
nclasses = f:read("nclasses"):all():long()[1]
nfeatures = f:read("nfeatures"):all():long()[1]

X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()
X_test = f:read("test_input"):all()

--total vocab plus padding
N = 37889+2
window_size = X_train:size(2)/2

caps = true

X_train, X_valid, X_test = capitalization(caps)

X_train = changeIndex(X_train, caps, window_size)
X_valid = changeIndex(X_valid, caps,window_size)
X_test = changeIndex(X_test, caps, window_size)

--We can concatenate when training on both train and valid
X_train = torch.cat(X_train, X_valid, 1)
Y_train = torch.cat(Y_train, Y_valid, 1)

--one hot encodings
Y_train_hot = onehotencode(nclasses, Y_train)
--Y_valid_hot = onehotencode(nclasses, Y_valid)

timer = torch.Timer()

W = lookupweights(X_train, Y_train, caps, window_size)

alpha = 0.055
W, b = NB(Y_train_hot, W, alpha)

predictions = predict(X_test, W, b)
_, indices_pred = torch.max(predictions, 2)
--_, indices_true = torch.max(Y_valid_hot, 2)
--score = predict_score()
--print(score)

print('Time elapsed: ' .. timer:time().real .. ' seconds')
write2file("NB3.csv", indices_pred)

Time elapsed: 35.46667599678 seconds	
